In [46]:
import os
import glob
import json
import pandas as pd
import log_codes as log

directory_path = './logs/dut1_iv_tee/logs_client'
file_pattern = 'Client_dut*'

# Use glob to find all files matching the pattern
file_paths = glob.glob(os.path.join(directory_path, file_pattern))

def parse_files(file_paths):
    frames = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            for json_line in file:
                # Parse frame
                try:
                    frame = json.loads(json_line)
                except Exception as error:
                    print(f"Failed decoding JSON for file {file_path}: {error}")
                frames.append(frame)
    return frames

frames_original = sorted(parse_files(file_paths), key=lambda x: x["timestamp"])
raw_df = pd.DataFrame(frames_original)

Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231209-094359.log.2023-12-09_18-51: Unterminated string starting at: line 1 column 99 (char 98)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231209-221707.log: Expecting value: line 1 column 1 (char 0)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231208-201708.log: Expecting value: line 1 column 1 (char 0)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231209-094359.log: Expecting value: line 1 column 1 (char 0)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231209-221709.log: Expecting value: line 1 column 1 (char 0)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231208-173416.log: Expecting value: line 1 column 1 (char 0)
Failed decoding JSON for file ./logs/dut1_iv_tee/logs_client/Client_dut1_monitor_20231209-181708.log:

### Converting Unix timestamp to UTC

And droping any duplicates.... Why do we do this though?????

In [47]:
raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'], unit='s').dt.tz_localize('UTC')
# raw_df = raw_df.drop_duplicates()
raw_df

,message,type,id,timestamp,data,event
0,,Serial dut1_monitor,20,2023-12-08 17:34:16.940597248+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
1,,Serial dut1_monitor,20,2023-12-08 17:34:18.877620224+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
2,,Serial dut1_monitor,20,2023-12-08 17:34:20.818251520+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
3,,Serial dut1_monitor,20,2023-12-08 17:34:22.755662336+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
4,,Serial dut1_monitor,20,2023-12-08 17:34:24.697770752+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
...,...,...,...,...,...,...
61952,,Serial dut1_monitor,20,2023-12-10 06:31:39.964876032+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
61953,,Serial dut1_monitor,20,2023-12-10 06:31:41.900495872+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
61954,,Serial dut1_monitor,20,2023-12-10 06:31:43.842961152+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN
61955,,Serial dut1_monitor,20,2023-12-10 06:31:45.778870016+00:00,aa013c000000004806c005b38bab004538f80066424602...,NaN


Parsing the fields based on frame type and then parsing the data tuple into readable fields

In [48]:
# Parse raw frame into executions
execution_df = log.parse_data(raw_df)
try:
    execution_df['timestamp'] = pd.to_datetime(execution_df['timestamp']).dt.tz_localize('UTC')
except Exception as error:
    print(f"Error converting timestamp to datetime: {error}")

execution_df[1:2]

,event,timestamp,total_errors,mcycle,minstret,ir_c,wait_ii,wait_if,wait_mc,load,store,wait_ls,branch,tbranch,imem_ecc,dmem_ecc,regfile,iv
1,0,2023-12-08 17:34:18.877620224+00:00,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0,0


-------------

# Data Analysis

Using ```execution_df``` as input

### total_errors

Filter for total errors and event transitions. This should give us a good idea of state transitions signifying SEEs.

In [71]:
# Ignore clean runs
filtered = execution_df[(execution_df['total_errors'] > 0) | (execution_df['event'] != 0)]

filtered_clean = execution_df[(execution_df['total_errors'] == 0) & (execution_df['event'] == 0)]

mean_filtered_clean = filtered_clean.iloc[:, 2:].mean()

# This line of code keeps only the rows where at least one of these values is different from the previous row's value.
filtered = filtered[~((filtered['mcycle'] == filtered['mcycle'].shift()) & (filtered['minstret'] == filtered['minstret'].shift()) & (filtered['event'] == filtered['event'].shift()))]

# This line of code keeps only the rows where the total_errors value is different from the previous row's value.
# filtered = filtered[~((filtered['total_errors'] == filtered['total_errors'].shift()))]

# This line of code keeps only the rows where the total_errors value is greater than 0.
filtered3 = filtered[(filtered['total_errors'] > 0)]

# Make a copy of filtered3 to avoid modifying the original DataFrame
filtered3_subtracted = filtered3.copy()
display(mean_filtered_clean)
# Subtract the mean values from filtered3, starting from the third column
filtered3_subtracted.iloc[:, 2:] -= mean_filtered_clean.values
filtered3_subtracted.iloc[:, 2:] = filtered3_subtracted.iloc[:, 2:].astype(int)

# display(filtered3[0:2])
# display(filtered3_subtracted[0:2])
filtered3_subtracted
# print(execution_df.iloc[2])
# filtered3

total_errors    0.000000e+00
mcycle          9.647061e+07
minstret        1.124242e+07
ir_c            1.626733e+07
wait_ii         3.815895e+07
wait_if         2.206329e+07
wait_mc         4.381079e+06
load            1.426024e+06
store           1.619628e+07
wait_ls         8.177561e+05
branch          5.095630e+05
tbranch         3.972329e+04
imem_ecc        8.317274e-02
dmem_ecc        2.743039e+03
regfile         1.993573e+04
iv              0.000000e+00
dtype: float64

/tmp/ipykernel_14924/1819000892.py:21: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  filtered3_subtracted.iloc[:, 2:] -= mean_filtered_clean.values
/tmp/ipykernel_14924/1819000892.py:22: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  filtered3_subtracted.iloc[:, 2:] = filtered3_subtracted.iloc[:, 2:].astype(int)


,event,timestamp,total_errors,mcycle,minstret,ir_c,wait_ii,wait_if,wait_mc,load,store,wait_ls,branch,tbranch,imem_ecc,dmem_ecc,regfile,iv
1131,0,2023-12-08 18:10:50.409622528+00:00,2,53376,6481,15578,28501,3009,1969,911,11459,563,375,-11193,159,-2743,-9546,0
3954,0,2023-12-08 19:46:40.309000448+00:00,2,-27764142,-3033884,-10545930,-17397178,-189080,-932415,-354537,-6189435,-313202,-164367,16116,0,-2743,27033,0
5293,0,2023-12-08 20:17:28.918361344+00:00,2,-27763549,-3032523,-10545546,-17398426,-192086,-932605,-354280,-6189730,-313243,-165167,170568,0,-2743,179404,0
5294,0,2023-12-08 20:17:30.304301312+00:00,2,-27764142,-3033884,-10545930,-17397178,-189080,-932415,-354537,-6189435,-313202,-164367,170224,0,-2743,179061,0
6453,0,2023-12-08 21:00:23.132166144+00:00,2,-27764142,-3033884,-10545930,-17397178,-189080,-932415,-354537,-6189435,-313202,-164367,225338,0,-2743,181222,0
15727,0,2023-12-09 02:04:16.574101760+00:00,1,-40508,-6074,-3224,49,-17282,-827,-1394,-139,-179,-451,-39493,2074,-2743,-19725,0
23339,0,2023-12-09 06:11:57.364787968+00:00,2,46735,5262,16060,26181,5059,1497,482,7804,510,255,-39723,159,-2743,-19935,0
27513,0,2023-12-09 08:28:38.782985216+00:00,2,136716,18826,31122,73779,-552,6087,3128,36525,1530,365,122648,0,-2743,61959,0
29262,0,2023-12-09 09:27:32.816411136+00:00,1,1271,280,-183,-431,987,-48,25,-121,0,-211,-3518,0,-2743,13827,0
35087,0,2023-12-09 12:39:08.061932032+00:00,2,41689,5777,6942,15032,4935,1891,956,8446,369,74,-38037,236,-2743,-19405,0
